# Part 1

In [1]:
def multiply_first_second(circular_list):
    return circular_list[0] * circular_list[1]


def run_through(input_lengths, circular_list, current_position=0, skip=0):
    for il in input_lengths:
        circular_list = tie_knot(circular_list, il, current_position)
        current_position = (current_position + il + skip) % len(circular_list)
        skip += 1
    return circular_list, current_position, skip


def tie_knot(cl, length, current_pos=0):
    assert length <= len(cl), f"invalid length: {length}, compared to {len(cl)}"
    newl = cl[current_pos:] + cl[:current_pos]
    newl[:length] = list(reversed(newl[:length]))
    return newl[len(cl)-current_pos:] + newl[:len(cl)-current_pos]

## Test Case from Description

In [2]:
multiply_first_second(run_through([3, 4, 1, 5], list(range(5)))[0])

12

In [3]:
assert tie_knot([0, 1, 2, 3, 4], length=3, current_pos=0) == [2, 1, 0, 3, 4]

tie_knot([0, 1, 2, 3, 4], length=3, current_pos=0)

[2, 1, 0, 3, 4]

In [4]:
assert tie_knot([2, 1, 0, 3, 4], length=4, current_pos=3) == [4, 3, 0, 1, 2]

tie_knot([2, 1, 0, 3, 4], length=4, current_pos=3)

[4, 3, 0, 1, 2]

In [5]:
assert tie_knot([4, 3, 0, 1, 2], length=1, current_pos=3) == [4, 3, 0, 1, 2]

In [6]:
assert tie_knot([4, 3, 0, 1, 2], length=5, current_pos=2) == [1, 0, 3, 4, 2]

tie_knot([4, 3, 0, 1, 2], length=5, current_pos=2)

[1, 0, 3, 4, 2]

In [7]:
assert tie_knot([4, 3, 0, 1, 2], length=5, current_pos=1) == [3, 4, 2, 1, 0]

tie_knot([4, 3, 0, 1, 2], length=5, current_pos=1)

[3, 4, 2, 1, 0]

## Puzzle Input Solution

In [8]:
with open('day/10/input') as fh:
    input_lengths = [int(length) for length in fh.read().strip().split(",")]

multiply_first_second(run_through(input_lengths, list(range(256)))[0])

19591

## Developing step-wise

In [9]:
al = [1, 2, 3, 6, 7, 8]

al

[1, 2, 3, 6, 7, 8]

In [10]:
pos = 2
length = 2

In [11]:
newl = al[pos:] + al[:pos]
newl

[3, 6, 7, 8, 1, 2]

In [12]:
newl[:length] = list(reversed(newl[:length]))
newl

[6, 3, 7, 8, 1, 2]

In [13]:
newl[len(al)-pos:] + newl[:len(al)-pos]

[1, 2, 6, 3, 7, 8]

# Part 2

- Take in a sequence of string chars, convert them to their ascii codes, add the suffix
- run the `run_through` function on this sequence 64 times, preserving the sequence, position, skip value between "rounds"
- convert the resulting re-shuffled list to a "dense hash"

In [14]:
def convert_to_ascii(iterable):
    return [ord(c) for c in iterable]

assert convert_to_ascii("1,2,3") == [49,44,50,44,51]


def make_lengths(input_str):
    return convert_to_ascii(input_str) + [17, 31, 73, 47, 23]

assert make_lengths("1,2,3") == [49,44,50,44,51,17,31,73,47,23]

In [15]:
from itertools import accumulate
from more_itertools import last, windowed
from operator import xor


def make_dense_hash(sparse):
    return [
        last(accumulate(grp, xor)) for grp in windowed(sparse, 16, step=16)
    ]

assert make_dense_hash(range(256)) == [0] * 16
assert last(accumulate([65, 27, 9, 1, 4, 3, 40, 50, 91, 7, 6, 0, 2, 5, 68, 22], xor)) == 64

def convert_to_hex(dense_hash):
    return "".join("{:02x}".format(n) for n in dense_hash)

assert convert_to_hex([64, 7, 255]) == '4007ff'

In [16]:
def compute_hash(some_string):
    input_lengths = make_lengths(some_string)
    pos = skip = 0
    circular_list = list(range(256))

    for _ in range(64):
        circular_list, pos, skip = run_through(input_lengths, circular_list, pos, skip)
    
    return convert_to_hex(make_dense_hash(circular_list))

assert compute_hash("") == "a2582a3a0e66e6e86e3812dcb672a272"
assert compute_hash("1,2,3") == "3efbe78a8d82f29979031a4aa0b16a9d"
assert compute_hash("1,2,4") == "63960835bcdc130f0b66d7ff4f6a5a8e"

## Solving the Puzzle

In [20]:
with open("day/10/input") as fh:
    input_lengths = fh.read().strip()

compute_hash(input_lengths)

'62e2204d2ca4f4924f6e7a80f1288786'

## What's Wrong with this?

Total mystery why this doesn't work, but since the puzzle is solved, keeping this only for the record.

In [17]:
assert compute_hash("AoC 2017") == "a2582a3a0e66e6e86e3812dcb672a272"

compute_hash("AoC 2017")

AssertionError: 

In [19]:
# Make sure `ord` does indeed convert stuff to ascii

x = "AoC 2017"
assert list(x.encode("ascii")) == [ord(n) for n in x]